![Logo de AA1](logo_AA1_texto_small.png) 
# Sesión 27 - Preprocesado con atributos de tipos diferentes

A lo largo de todas las sesiones prácticas hemos estado utilizando diferentes algoritmos que nos han permitido trabajar con los datos. Podemos clasificar estos algoritmos como *Transformadores* o *Estimadores*:
- **Tansformadores**. Un *transformador* es un objeto que tiene los métodos `fit()` y `transform()` y suele utilizarse para el preprocesado de los datos. Los transformadores pueden utilizarse para la limpieza de los datos, el escalado de atributos, la reducción de la dimensionalidad,...
- **Estimadores**. Un *estimador* es un objeto que es capaz de aprender a partir de los datos y generar un modelo con capacidades predictivas. Estos objetos tienen los métodos `fit()` y `predict()`.

Los conjuntos de datos que hemos utilizado hasta el momento han sido conjuntos de datos en los que todos los atributos se preprocesaban de la misma manera o se daban circunstancias que nos permitían preprocesarlos de manera manual.

Sin embargo, en los conjuntos de datos reales podemos encontrarnos con que los atributos necesitan preprocesarse de manera diferente. Por ejemplo, los atributos de tipo categórico querremos codificarlos mediante una codificación one-hot, los atributos numéricos podemos querer escalarlos de alguna manera y los atributos binarios puede ser que queramos dejarlos tal y como están.

También se puede dar el caso de que tengamos valores desconocidos y que necesitemos *imputar* valores. Esto se hará de manera diferente dependiendo del tipo de atributo de que se trate.

Y, por supuesto, debemos tener en cuenta también que los transformadores deben realizar el `fit()` sobre el conjunto de entrenamiento, ya que si lo hacemos sobre todo el conjunto de datos estaríamos utilizando los datos de test durante el proceso de entrenamiento y eso no es correcto.

Para solucionar este último punto hemos utilizado los `Pipeline` y nos han resultado muy útiles puesto que podíamos enlazar por ejemplo, un `StandardScaler` y una `SVR`. Sin embargo, un `Pipeline` creado de tal manera aplicaría el `StandarScaler` a todos los atributos, incluso a los categóricos si los hubiese.

## 27.1 Preprocesado de atributos diferenciado

Dentro de `Scikit-learn` podemos encontrar una herramienta muy útil para el tratamiento deferenciado de grupos de atributos. Esa herramienta es el `ColumnTransformer` y vamos a ir viendo poco a poco cómo se utiliza.

Para empezar vamos a cargar un conjunto de datos que necesita preprocesar unos atributos de una manera y otros de otra manera:


In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVR

# los valores están separados por uno a varios espacios en blanco
# hay valores desconocido identificados como '?'
cabecera = ['class (mpg)','cylinders','displacement','horsepower','weight','acceleration', 'model year', 'origin', 'car name']
df = pd.read_csv('auto-mpg.data', sep='\s+', names=cabecera, na_values='?')
display(df)

,class (mpg),cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.0,2130.0,24.6,82,2,vw pickup
395,32.0,4,135.0,84.0,2295.0,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.0,2625.0,18.6,82,1,ford ranger


En el fichero **auto-mpg.names** se describen los atributos de la siguiente manera:

1. mpg:           continuous
2. cylinders:     multi-valued discrete
3. displacement:  continuous
4. horsepower:    continuous
5. weight:        continuous
6. acceleration:  continuous
7. model year:    multi-valued discrete
8. origin:        multi-valued discrete
9. car name:      string (unique for each instance)

*mpg* se refiere a "miles per gallon", es la clase que se desea aprender. En este conjunto se presentan características de diferentes modelos de coches y lo que se pretende es ver si se puede predecir el consumo del coche en función de esos atributos.

El último atributo, *car name*, es el nombre del modelo de coche y no aporta información relevante, así que vamos a eliminarlo.

In [2]:
# se elimina car name porque no aporta información al problema
df = df.drop(columns=['car name'])
display(df)

,class (mpg),cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1
...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1


Los atributos *displacement*, *horsepower*, *weight* y *acceleration* vienen marcados como continuos en el fichero names.

Se indica que los atributos *cylinders* y *model year* son *multi-valued discrete*. Si observamos los datos vemos que el primero indica el número de cilindros y que tiene un orden ($4 \lt 6 \lt 8$), así que podemos considerarlo también de tipo numérico. *model year* indica el año de lanzamiento del modelo al mercado y tiene también un sentido numérico. Podríamos considerarlos ordinales, pero al ser números y tener relevancia su orden vamos a considerarlos como atributos de tipo numérico.

El atributo *origin* tiene 3 posibles valores que se corresponden con el origen de la marca del coche:
- 1 -> América
- 2 -> Europa
- 3 -> Japón

Así que este atributo es de tipo categórico.

Por tanto, **tenemos un atributo de tipo categórico y el resto de tipo numérico y deben ser tratados de manera diferente en el preprocesado de datos**.

Con el fin de poder visualizar mejor los pasos que vamos dando, en un principio vamos a utilizar únicamente 5 ejemplos a modo prueba. Más adelante ya utilizaremos todo el conjunto:


In [3]:
# se barajan los ejemplos y nos quedamos solo con los 5 primeros
df5 = df.sample(random_state=2, n=5)

# la clase está en la primera columna!
# separamos las últimas columnas y las almacenamos en X
X = df5.iloc[:,1:]
# X = df.drop(['class (mpg)'], axis=1)

# separamos la clase
y = df5.iloc[:,0]
# y = df['class (mpg)'] 

print('\n##########################################')
print('### Hold-out 80-20')
print('##########################################')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
print("X_train:")
display(X_train)
print("X_test:")
display(X_test)
print("y_train:")
display(y_train)
print("y_test:")
display(y_test)


##########################################
### Hold-out 80-20
##########################################
X_train:


,cylinders,displacement,horsepower,weight,acceleration,model year,origin
94,8,440.0,215.0,4735.0,11.0,73,1
32,4,98.0,NaN,2046.0,19.0,71,1
279,4,98.0,68.0,2135.0,16.6,78,3
178,4,120.0,88.0,2957.0,17.0,75,2


X_test:


,cylinders,displacement,horsepower,weight,acceleration,model year,origin
354,4,100.0,NaN,2320.0,15.8,81,2


y_train:


94     13.0
32     25.0
279    29.5
178    23.0
Name: class (mpg), dtype: float64

y_test:


354    34.5
Name: class (mpg), dtype: float64

Para el barajado hemos utilizado una semilla de aletorios que selecciona 5 ejemplos adecuados para lo que queremos mostrar (por eso no hemos utilizado la semilla que utilizamos habitualmente).

Vamos ahora a crear grupos de atributos. Antes vimos que salvo el atributo *origin* que es de tipo categórico, el resto de atributos es de tipo numérico, así que vamos a crear dos listas que identifiquen a los atributos de ambos grupos:

In [4]:
# se identifican los grupos de atributos quese quieren preprocesar
atr_nume = X_train.columns.drop('origin')
atr_cate = ['origin']

Podría haber otros grupos de atributos, como por ejemplo los binarios, y también podría haber atributos a los que no se les quiera hacer ningún preprocesado.

Ahora vamos a definir lo que querríamos hacer con los atributos de cada uno de esos grupos utilizando un `Pipeline` para cada grupo.

Empezamos por los categóricos:

In [5]:
# definimos pipeline para atributos categóricos
pipe_cate = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('oh', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

En este `Pipeline` estamos indicando que queremos que a los atributos categóricos primero se les asigne un valor a sus valores desconocidos. En este caso estamos indicando que si no se conoce el lugar de origen de la marca que se ponga un 0, que es una manera de decir que no es de los lugares conocidos.

Posteriormente se realiza una codificación one-hot. En este caso vamos a utilizar `OneHotEncoder`: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html?highlight=onehotencoder#sklearn.preprocessing.OneHotEncoder 

Vamos a probar el `Pipeline` para ver lo que hace, aunque más adelante lo utilizaremos de manera más sencilla:

In [6]:
# entrenamos el pipeline sobre la columna 'origin'
pipe_cate.fit(X_train['origin'].values.reshape(-1,1))

display(X_train)

# tranformamos la columna 'origin' de X_train
pipe_cate.transform(X_train['origin'].values.reshape(-1,1))

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
94,8,440.0,215.0,4735.0,11.0,73,1
32,4,98.0,NaN,2046.0,19.0,71,1
279,4,98.0,68.0,2135.0,16.6,78,3
178,4,120.0,88.0,2957.0,17.0,75,2


array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

Vemos que el pipeline funciona puesto que asigna una columna a cada valor posible y transforma los ejemplos correctamente.

Vamos ahora a hacer el pipeline para los atributos numéricos:

In [7]:
# definimos pipeline para atributos numéricos
pipe_nume = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std', StandardScaler())
])

En este caso se utiliza un *imputer* que asigna la mediana a los valores desconocidos y posteriormente estandariza.

Vamos a verlo funcionar en el atributo *horsepower*:


In [8]:
# entrenamos el pipeline sobre la columna 'horsepower'
pipe_nume.fit(X_train['horsepower'].values.reshape(-1,1))

display(X_train)

# tranformamos la columna 'horsepower' de X_train
pipe_nume.transform(X_train['horsepower'].values.reshape(-1,1))

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
94,8,440.0,215.0,4735.0,11.0,73,1
32,4,98.0,NaN,2046.0,19.0,71,1
279,4,98.0,68.0,2135.0,16.6,78,3
178,4,120.0,88.0,2957.0,17.0,75,2


array([[ 1.71506961],
       [-0.45763703],
       [-0.79979555],
       [-0.45763703]])

Lo primero que ha hecho ha sido asignar la mediana (88) al valor desconocido y después ha estandarizado los valores resultando los valores que se muestran.

Ya hemos visto que ambos pipelines funcionan de forma correcta si les damos los atributos adecuados. Ahora es cuando entra en juego el `ColumnTransformer` para facilitarnos la vida: https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

In [9]:
print("Atributos numéricos:", atr_nume)
print("Atributos categóricos:", atr_cate)

# definimos un ColumnTransformer para tratar los atributos de manera diferente
# y los que no pertenezcan a ninguno de los grupos se deja tal cual está
at_transformer = ColumnTransformer([
    ('at_cat', pipe_cate, atr_cate),
    ('at_num', pipe_nume, atr_nume)], 
    remainder='passthrough'
)

Atributos numéricos: Index(['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration',
       'model year'],
      dtype='object')
Atributos categóricos: ['origin']


Le hemos indicado que a los atributos categóricos les aplique el pipeline `pipe_cate` y a los numéricos el pipeline `pipe_nume`. Si hubiese más atributos que no perteneciesen a ninguno de esos dos grupos no se haría nada con ellos (`remainder='passthrough'`).

**IMPORTANTE:** En el transformer hemos situado el pipeline de los atributos categóricos primero y el de los numéricos después. Esto implica que tras aplicar el transformer aparecerán primero los atributos categóricos y después los numéricos. Además, seguirán el orden que hayamos indicado en las listas correspondientes. 

Ahora podemos entrenar este *transformer* y ver su resultado:

In [10]:
# lo entrenamos
at_transformer.fit(X_train)

# para comprender mejor la salida, vamos a mostrar los datos dentro de un dataframe con los nombres adecuados
# Los atributos categóricos en formato one-hot se nombrará como 'nombreAtributo_valorAtributo'
nombres_oh_atr_cat = at_transformer.named_transformers_['at_cat']['oh'].get_feature_names_out(atr_cate)
nombres_columnas = np.append(nombres_oh_atr_cat, atr_nume)
print(nombres_columnas)

print("####################### Conjunto de entrenamiento #######################")
# mostramos los datos originales
display(X_train)
#mostramos los datos transformados (dentro de un dataframe)
display(pd.DataFrame(at_transformer.transform(X_train), columns=nombres_columnas))

print("####################### Conjunto de test #######################")
# mostramos los datos originales
display(X_test)
#mostramos los datos transformados (dentro de un dataframe)
display(pd.DataFrame(at_transformer.transform(X_test), columns=nombres_columnas))

['origin_1' 'origin_2' 'origin_3' 'cylinders' 'displacement' 'horsepower'
 'weight' 'acceleration' 'model year']
####################### Conjunto de entrenamiento #######################


,cylinders,displacement,horsepower,weight,acceleration,model year,origin
94,8,440.0,215.0,4735.0,11.0,73,1
32,4,98.0,NaN,2046.0,19.0,71,1
279,4,98.0,68.0,2135.0,16.6,78,3
178,4,120.0,88.0,2957.0,17.0,75,2


,origin_1,origin_2,origin_3,cylinders,displacement,horsepower,weight,acceleration,model year
0,1.0,0.0,0.0,1.732051,1.728734,1.715070,1.635742,-1.648981,-0.483368
1,1.0,0.0,0.0,-0.577350,-0.626752,-0.457637,-0.853864,1.043233,-1.256757
2,0.0,0.0,1.0,-0.577350,-0.626752,-0.799796,-0.771463,0.235569,1.450105
3,0.0,1.0,0.0,-0.577350,-0.475230,-0.457637,-0.010416,0.370179,0.290021


####################### Conjunto de test #######################


,cylinders,displacement,horsepower,weight,acceleration,model year,origin
354,4,100.0,NaN,2320.0,15.8,81,2


,origin_1,origin_2,origin_3,cylinders,displacement,horsepower,weight,acceleration,model year
0,0.0,1.0,0.0,-0.57735,-0.612977,-0.457637,-0.600181,-0.033653,2.610189


Vemos que los conjuntos de entrenamiento y test se han generado de manera correcta realizando preprocesados diferentes en cada atributo en función de lo que le hemos indicado.

Estas tranformaciones las hemos hecho nosotros llamando a `fit()` y a `transform()` por motivos didácticos para que viésemos lo que va sucediendo paso a paso. 

En la práctica podemos hacer que todo sea más automático utilizando un pipeline que realice las transformaciones y luego llame a un estimador:

In [11]:
# pipeline que realiza el preprocesado y lo encadena con un estimador
sys_prep_svr = Pipeline([
    ('atr_trans', at_transformer),
    ('sys', SVR())
])

# entrenamos
sys_prep_svr.fit(X_train, y_train)

# Realizamos una predicción sobre el mismo conjunto de entrenamiento
y_train_pred = sys_prep_svr.predict(X_train)
print("Predicciones en el conjunto de entrenamiento:", y_train_pred)

# Realizamos una predicción sobre el conjunto de test
y_test_pred = sys_prep_svr.predict(X_test)
print("Predicciones en el conjunto de test:         ", y_test_pred)

Predicciones en el conjunto de entrenamiento: [22.62302794 24.40201337 24.2983633  23.59798664]
Predicciones en el conjunto de test:          [23.86682865]


Como vemos, hemos creado un sistema que realiza todo el proceso de preprocesado y generación del modelo utilizando `ColumnTranformer` y `Pipeline` convenientemente anidados.

En la siguiente figura podemos ver cómo será el flujo de datos en el sistema que hemos creado:

![Sistema](fig_sistema.png) 


## 27.2 Podemos utilizarlo como un sistema más

Hemos creado un sistema un poco más complejo, pero una vez creado podemos utilizarlo dentro de búsquedas de hiperparámetros, validaciones cruzadas,... lo que que queramos.

Vamos a ver un ejemplo de uso utilizando todos los datos (no solo 5). Así que vamos a separar los atributos de la clase y luego vamos a separar los ejemplos en entrenamiento y test:

In [12]:
print('\n##########################################')
print('### Cogemos todos los datos')
print('##########################################')

# separamos las últimas columnas y las almacenamos en X
X = df.iloc[:,1:]

# separamos la clase
y = df.iloc[:,0]

print('\n##########################################')
print('### Hold-out 80-20')
print('##########################################')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)


##########################################
### Cogemos todos los datos
##########################################

##########################################
### Hold-out 80-20
##########################################


Ahora vamos crear una búsqueda de hiperparámetros en la que vamos a utilizar el sistema `sys_prep_svr` y vamos a buscar los mejores valores para `C` y para `gamma`, que son los hiperparámetros más importantes cuando estamos utilizando una Máquina de Vectores Soporte (SVM) de Regresión (`SVR`) con kernel `rbf`:

In [13]:
# se crea un generador de folds partiendo el conjunto en 5 trozos
folds = KFold(n_splits=5, shuffle=True, random_state=1234)

# creamos una grid search para el SVC donde le pasamos los hiperparámetros que queremos probar
param = {
    'sys__C': [0.01, 0.1, 1, 10, 100],
    'sys__gamma': [0.01, 0.1, 1, 10, 100]
}
gs = GridSearchCV(sys_prep_svr, param_grid=param, scoring='neg_mean_squared_error', cv=folds, verbose=1, n_jobs=-1)

Realizamos la búsqueda y obtenemos el modelo con los mejores hiperparámetros:

In [14]:
# entrenamos
best_model = gs.fit(X_train, y_train)

print("Mejor combinación de hiperparámetros:", best_model.best_params_)
print("Mejor rendimiento obtenido: %.4f" % best_model.best_score_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Mejor combinación de hiperparámetros: {'sys__C': 100, 'sys__gamma': 0.01}
Mejor rendimiento obtenido: -8.1408


Recordad que el rendimiento aparece en negativo porque utilizamos `'neg_mean_squared_error'`. 

Ya tenemos el modelo entrenado y podemos obtener su rendimiento ante casos no vistos en el entrenamiento:

In [15]:
# obtenemos su rendimiento en el conjunto de test
y_pred = best_model.predict(X_test)
print("MSE:", metrics.mean_squared_error(y_test, y_pred))
print("MAE:", metrics.mean_absolute_error(y_test, y_pred))
print("R2 :", metrics.r2_score(y_test, y_pred))

MSE: 6.452866157394412
MAE: 1.8059202450275507
R2 : 0.8834032248254028


## Ejercicios

1. Carga el conjunto de datos  **CreditApproval.data** 
2. Selecciona 5 ejemplos y vete realizando las codificaciones paso a paso como hemos visto en la práctica. Que no os asuste ver datos con caracteres, serán atributos categíricos o binarios y se tratan como ya hemos visto.
3. Para los atributos binarios se puede utiliza `OneHotEncoder()` utilizando el parámetro `drop` convenientemente. La clase se codifica al principio, nada más cargar los datos.
4. Una vez que veas que la codificación funciona haz una búsqueda de hiperparámetros utilizando todos los ejemplos

Estos ejercicios no es necesario entregarlos.